# Non Deterministic Automaton

In [1]:
class NonDeterminsiticAutomaton:
    
    def __init__(self, q = None, sigma = None, delta = None, q0 = None, empty_symbol = "lambda",f = None):
        """
        Class initializar
        
        self.q receives all the states that conform the dfa
        
        self.sigma are all the variables defined in the dfa
        
        self.delta all are the operation defined, i.e- is a dictionary contating a tuple as key
        where the left values of the tuple is the current state and the right value the variable pass and the value obtain by
        accessing to with the key is to which state goes.
        
        self.q0 is the initial state
        
        self.empty_symbol is the symbol in which is represent the empty value, for instance, it is lambda bout could be epsilon
        or another symbol, but it has to be defined, it is not receive from txt.
        
        self.f are all the final states
        
        self.actual_state is the state which the dfa finish after proccessing a word, if no word proccess the actual state is q0
        and it is reset at start proccessing a new word
        """
        self.q = q
        self.sigma = sigma
        self.delta = delta
        self.q0 = q0
        self.empty_symbol = empty_symbol
        self.f = f
        self.actual_state = q0
        
        self.not_valid_actual_state = q0
        
        #private attributes
        self.set_empty_transictions = []
        
        self.all_reacheble_states = set()
        
        self.valid_paths = []
        
        self.not_valid_paths = []
        
        #boolean to know if a word is accepted or not
        self.trampa = False
        
        self.all_paths_for_a_symbol = []
        
        self.all_paths_empty_transitions = []
        
        
    def load_from_file(self, path, delta_separator = "-"):
        """
        Function to read a file that contains an automaton
        
        path: path file
        """
        
        #dictionary to save the parts of the automaton
        variables = {"q": None,
             "sigma": None,
             "delta": None,
             "q0": None,
             "f": None    
        }

        with open(path) as file:
            #parsing the document
            for line, variable in zip(file, variables):
                    line = line.replace("{", "")
                    line = line.replace("}", "")
                    line = line.replace("\n", "")
                    line = line.replace(" ", "")
                    value = list(line.split(","))
                    variables[variable] = value
                    
            #conditional to verify there is only one initial state        
            if len(variables["q0"]) > 1 or len(variables["q0"]) == 0:
                raise Exception('There are more than one initial state')
                
            variables["delta"] = [values.split(delta_separator) for values in variables["delta"]]
            
            #Memory to check that within the transictions there is no variable with more than one variable
            Memory = [True for value in variables["delta"] if value[1] not in variables["sigma"] and value[1] != self.empty_symbol]
            
            
            if sum(Memory) >= 1:
                # for example the language is {a, b} and the variable of that transiction is aa.
                raise TypeError("At least One variable has a transiction with a variable that does not belong to sigma")
                
            #creating all posible combination of each state with each symbol
            all_possible_transictions = {(state,symbol): ["z0_dead"] for state in variables["q"] for symbol in variables["sigma"] }
            
            #creating transitions from the state of dead
            for symbol in variables["sigma"]:
                 all_possible_transictions[("z0_dead",symbol)] = ["z0_dead"]
            
            

            #parsing
            #getting the values from delta
            for values in variables["delta"]:
                #creating the key
                key = (values[0],values[1])
                
                #checking whether the key is 
                if key in all_possible_transictions.keys():
                    #if the first variables is z0_dead, so that is the defaul creation and actually has transitions
                    if all_possible_transictions[key][0] == "z0_dead":
                        all_possible_transictions[key].pop()
                        all_possible_transictions[key].append(values[2])
                        
                    else:
                        all_possible_transictions[key].append(values[2])

                else:
                    all_possible_transictions[key] = [values[2]]
                        
             #getting q0
            variables["q0"] = variables["q0"][0]
            
            #assigning the attributos of the class to the values obtained from the file
            self.q = variables["q"]
            self.sigma = variables["sigma"]
            self.delta = all_possible_transictions
            self.q0 = variables["q0"] 
            self.f = variables["f"]
            self.actual_state = variables["q0"]
            
            #conditional to verify the dfa is valid
            if self.is_valid():
                pass
            else:
                raise Exception('Is not a valid automaton')
            
            
    
    def empty_transitions(self, state, showing_backtracking = True):
        """
        Function to find all reachable states from an initial state just passing empty symbols
        
        Input:
        Current State
        showing_backtracking: boolean to know whether to print the path or not
        
        Output:
        return a list of all reachable state
        """
        if set([state]).issubset(set(self.q)):
            pass
        else:
            raise Exception("The given state does not belong to this automaton")
        
        #reseting the set of empty transiction, this is a private attribute that is only useful in the methods of the classs
        self.set_empty_transictions = set()
        
        #creating a list to save the backtracking
        paths = [state]
        
        
        self.empty_transitions_recursive(state, [state], paths, showing_backtracking) 
        
        try:
            #if exists, then discard it
            self.set_empty_transictions.discard("z0_dead")
            return list(self.set_empty_transictions)
            
        except:                
            return list(self.set_empty_transictions)
        
        
        
    def empty_transitions_recursive(self, state, memory = [], paths = [], showing_backtracking = True):
        """
        Private Function to find all reachable states from an initial state just passing empty symbols, recursively
        we did it in this way so we can get print the path, I mean for empty transitions, getting the paths may be
        useless because it works for nothing but it seems neat xD.
        
        Input:
        state: Current State
        memory: is a list of states already visited by that path
        showing_backtracking: boolean to know whether to print the path or not
        
        Output:
        return a list of all reachable state
        """
    
        #memory to avoid infinite loops
        memory_inside = memory[:]
        
        memory_inside.append(state)
        
        if showing_backtracking:
            print(paths)
        
        #check if that transiction exists
        try:
            for new_state in self.delta[(state, self.empty_symbol)]:
                #to avoid infinite loops
                if new_state not in memory:
                    #backtracking
                    paths.append(new_state)
                    
                    #adding the new_state into that set
                    self.set_empty_transictions.add(new_state)
                    #recursion 
                    self.empty_transitions_recursive(new_state, memory_inside, paths, showing_backtracking)
                    
                    #eliminating the element, for backtracking
                    paths.pop()
                    
                else:
                    #when the infinite loop exist
                    paths.append(new_state)
                    
                    if showing_backtracking:
                        print(paths)
                        
                    #eliminating the element, for backtracking
                    paths.pop()
                    
        except:
            pass
            
        
        
        
    def process_symbol(self, state, symbol, showing_backtracking = True):
        """
        Function to find all the states you can reach from one state processing a symbol
        
        Input:
        state: current state
        
        symbol: variable belonging to sigma
        
        showing_backtracking: boolean to know whether to print the path or not
        
        Output:
        return a list of the reacheble states
        """
        self.valid_string(symbol)
        
        if set([state]).issubset(set(self.q)):
            pass
        else:
            raise Exception("The given state does not belong to this automaton")
        
        #reseting attribute
        self.all_paths_for_a_symbol = []
        
        #setting that set into the first reachable states
        #First case where only consumes the symbol
        self.all_reacheble_states  = set(self.delta[(state, symbol)])
        
        paths = [(state, symbol)]
        
        paths.append(self.delta[(state, symbol)])
        if showing_backtracking:
            print("-------------------------Case where reach a state only consuming the symbol-------------------------")
            print(paths)
            
        if self.delta[(state, symbol)] != []:
            self.all_paths_for_a_symbol.append(paths[:])
            
        paths.pop()
        
        
        #Second case where the symbol is consumed and then applyes lambda
        
        if showing_backtracking:
            print("-------------------------Case where reach a state consuming the symbol and then lambda if possible-------------------------")
        
        for new_state in self.delta[(state, symbol)]:
            
            if new_state != "z0_dead":
                #adding the state and the symbol for transiction
                paths.append((new_state, self.empty_symbol))
                

                #adding where it can reach after making that transiction
                empty_transitions_list = self.empty_transitions(new_state, False)
                
                paths.append(empty_transitions_list)
                
                if empty_transitions_list != []:
                    self.all_paths_for_a_symbol.append(paths[:])
                if showing_backtracking:
                    print(paths)

                #uptading the set
                self.all_reacheble_states.update(empty_transitions_list)
                paths.pop()
                paths.pop()
        
        
        #third case first consumbes the empty symbol and then the synbol 
        #and also fourth case where consumes the empty symbol, then the symbol and again the empty symbol
        #getting the transitions using lambda
        new_reacheable_states = self.empty_transitions(state, False)
        
        
        if showing_backtracking:
            print("-------------------------Third and Fourth Case-------------------------")
        
        paths = [(state, self.empty_symbol)]
        
        #going through the reacheable states using lambda and then the symbol
        for new_state in new_reacheable_states:
            
            #saving the symbol and the symbol consume
            paths.append((new_state, symbol))
            #adding the reacheable states
            paths.append(self.delta[(new_state, symbol)])
            if showing_backtracking:
                print(paths)
    
            self.all_reacheble_states.update(self.delta[(new_state, symbol)])
            
            self.all_paths_for_a_symbol.append(paths[:])
            #eliminating those reachebale states because where are going to iterate, one by one
            paths.pop()
            
            #going through the reacheable states using lambda, then the symbol and finally the empty transition
            for second_next_state in self.delta[(new_state, symbol)]:
                if second_next_state != "z0_dead":
                    #saving the symbol and the symbol consume
                    paths.append((second_next_state, self.empty_symbol))
                    #adding the reacheable states
                    
                    empty_transitions_list = self.empty_transitions(second_next_state, False)
                    
                        
                    paths.append(empty_transitions_list)
                    if showing_backtracking:
                        print(paths)
                    
                    if empty_transitions_list != []:
                        self.all_paths_for_a_symbol.append(paths[:])
    
                    self.all_reacheble_states.update(empty_transitions_list)

                    #eliminating for backtracking
                    paths.pop()
                    paths.pop()

            #eliminating for backtracking    
            paths.pop()
            
        lista_reacheable_state = list(self.all_reacheble_states)    
            
        if "z0_dead" in lista_reacheable_state:
            lista_reacheable_state.append(lista_reacheable_state.pop(lista_reacheable_state.index("z0_dead")))
            
        return lista_reacheable_state
    
    def process_word(self, word, showing_backtrack = True):
        """
        Function to determine if a word belong to the language of the ndfa
        
        input:
        Word: word to be proccessed
        
        Output:
        return a bolean whether if the word is proccessed by the automaton or not
        """ 
        self.valid_string(word)
        
        word = str(word)
         
        #reseting attributes
        self.trampa = False
        
        self.valid_paths = []
        
        self.not_valid_paths = []
        
        #memory
        paths = []
          
        #going into recursive
        self.process_word_recursive(word, self.q0 ,paths, showing_backtrack)
        
        return self.trampa
        
    
    def process_word_recursive(self, word, state,paths = [], showing_backtrack = True):
        """
        Private method for the method process_word, in which it recursively goes to each possible paths to try to find a possible path 
        where the word is accepted
        input:
        Word: word to be proccessed
        paths: list which is the memory for the backtracking
        showing_backtrack: shows the backtracking or not
        
        Output:
        return a bolean whether if the word is proccessed by the automaton or not
        """
        #check if the word is not empty
        try:
            letter = word[0]
        
        #case where is no more variables to consume
        except:
            #checking if the last state is in final states
            if state in list(self.f):
                if showing_backtrack:
                    print("---------------------------valid-----------------------\n\n")
                #saving that the word is valid   
                self.trampa = True
                #actul state to use in function str
                self.actual_state = state
                #saving the valid path
                self.valid_paths.append(paths[:])
                return 
            
            #unvalid word
            else:
                if showing_backtrack:
                    print("---------------------------Unvalid---------------------------\n\n")
                    #the same as the part latter
                self.not_valid_actual_state = state
                self.not_valid_paths.append(paths[:])
                return
        
        #adding to the backtracking
        paths.append((state,letter))
        if showing_backtrack:
            print(paths)
        #going through each possible state with that letter
        for possible_path in self.process_symbol(state, letter, False):
            #saving for backtracking
            paths.append(possible_path)
            if showing_backtrack:
                print(paths)
                
            #if it reaches the state of dead
            if possible_path == "z0_dead":
                if showing_backtrack:
                    print("---------------------------Unvalid---------------------------\n\n")
                paths.pop()
                paths.pop()
                #you may think this should be a pass, but no because our  state of death, thank to the function process_symbol
                # is always and the end of the list
                return
            
            self.process_word_recursive(word[1:], possible_path, paths, showing_backtrack)
            paths.pop()
        paths.pop()
        
    def valid_string(self, word):
        
        if set(word).issubset(set(self.sigma)):
            return
        else:
            raise Exception("Not valid word, contains variables that are not part of the language")
    
        
        
    def __str__(self):
        
        if len(self.valid_paths) == 0:
            self.actual_state = self.not_valid_actual_state
        
        cadena = "Grammar: \n q: {} \n sigma: {}, delta: {}, q0: {}, f: {}, current_sate: {}, empty_symbol: {}".format(
                                                                                    self.q , 
                                                                                    self.sigma, 
                                                                                    self.delta, 
                                                                                    self.q0, 
                                                                                    self.f, 
                                                                                    self.actual_state,
                                                                                    self.empty_symbol) 
        return cadena
    
    def is_valid(self):
        
        """
        Funtion to validate whether the dfa is valid or not
        
        Uses the attributes of the class
        """
        
        #gets the tuples which are the deltas defined
        all_defined_oper = [(key1,key2)for key1, key2 in self.delta.keys() if key1 != "z0_dead"]
        
        #get the values of the deltas
        all_to_go_states = [value for value in self.delta.values()]
        
        all_to_go_states = [item for sublist in all_to_go_states for item in sublist if item != "z0_dead"]
        
        #gets the left value of the tuple
        all_states_to_receive = set([state[0] for state in list(self.delta.keys()) if state[0] != "z0_dead"])
        
        #gets the right value of the tuple
        all_variables_to_receive = set([state[1] for state in list(self.delta.keys()) if state[0] != "z0_dead"])
        
        
        #conditional to check if the final states are subsets of q, which is the variable containing all the states
        if set(self.f).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("States of F are not a subset of states q")
            
            
        #conditional to check if the inital state is a subsets of q, which is the variable containing all the states    
        if set([self.q0]).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("q0 is not a subset of states q")
            
        
        #checking if the left value of the tuple is subset of q and the right value is subset of sigma
        if set(all_states_to_receive).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("The state before a applying the transiction is not a subset of q")
            
        sigmas = set(self.sigma)
        sigmas.add(self.empty_symbol)
        if set(all_variables_to_receive).issubset(sigmas):
            pass
        
        else:
            print(all_variables_to_receive)
            print(self.sigma)
            raise TypeError("The variable which is apply to the transiction is not a subset of sigma")
        
        #checking if all the transictions of delta are subsets of q
        if set(all_to_go_states).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("The state resultant of a transiction is not a subset of states q")
            
        return True
    
    def print_valid_path(self, path = None):
        """
        Function to print one path of an accepted word, only works with valid paths, can be obtained using self.valid_paths
        
        input:
        path: list that contains the path travelled
        
        output:
        None
        only prints the path
        """
        # if the path is not defined, it gets the first path of valid_paths
        if path == None:
            path = self.valid_paths[0]
            
        #saving the paths
        llaves = []
        caminos = []
        
        #this division is used to avoid problems and make it easier to use because a valid path always has a pair number of elements+
        #where the pair are a tuple of the transiction and the impairs is the resulf ot the tuple before
        n = int(len(path)/2)
        
        #going through this list
        for i in range(n):
            #process the symbol of the right side of the tuple in the state that is in the left side of the same tuple
            self.process_symbol(path[i*2][0],path[i*2][1], False)
            
            #going through each set of paths
            for j in range(len(self.all_paths_for_a_symbol)):
                #the list obtained in process symbol and saved in self.all_paths_for_a_symbol is made to each last element of the list within 
                #list are the states that reaches, so we get a path to reach that state consuming that symbol
                if path[i*2 + 1] in self.all_paths_for_a_symbol[j][-1]:
                    aux = self.all_paths_for_a_symbol[j]
                    #saving the values
                    llaves.append((path[i*2][0],path[i*2][1], path[i*2 + 1])) 
                    caminos.append(aux)
                    break 
        
        #we eliminate if there is more than one final state in the last element of the last list
        for i in caminos[-1][-1]:
            if i in self.f:
                caminos[-1][-1] = [i]
                break
        
        #This code is to clean the paths adding the exact lambda transiction
        n = len(caminos)
        for path in range(n):
            for trans in range(len(caminos[path])):
                #all the case before getting to the last element that always is an array
                if trans != len(caminos[path])-1:
                    #case where the transiction is lambda
                    if caminos[path][trans][1] == self.empty_symbol:
                        #finding the path
                        self.find_path(caminos[path][trans][0], caminos[path][trans+1][0])
                        #case where the transiction is before the last element
                        if trans == len(caminos[path])-2:
                            caminos[path].pop(trans)
                            caminos[path].pop(trans)
                            #adding the transictions using lambda to get to the last element of the list
                            for val in self.all_paths_empty_transitions[::-1]:
                                    caminos[path].insert(trans, val)  
                        #case where the element after the transictions is another tuple
                        else:
                            
                            caminos[path].pop(trans)
                            #adding the elements except the last one because is just a list, and is worthless
                            for val in self.all_paths_empty_transitions[1::-1][1:]:
                                    caminos[path].insert(trans, val) 
                            
        
        i = 1
        #loops where the printins is made
        j = 0
        for values in list(caminos):
            print("{}.- ".format(i), end = "")
            for transition in range(len(values)):
                if transition < (len(values)-1):
                    if len(values[transition+1]) > 1 and type(values[transition+1]) == list:
                        for val in values[transition+1]:
                            if j < len(caminos) - 1:
                                if val in caminos[j+1][0]:
                                    print("{}  ======> {}---- ".format(values[transition],val), end = "")  
                            else:
                                if val in self.f:
                                    print("{}  ======> {}---- ".format(values[transition],val), end = "")
                                    break
                    else:
                        print("{}  ======> {}---- ".format(values[transition],values[transition+1][0]), end = "")

                else:
                    print()
            j += 1
            i += 1
    
    def find_path(self, state, reach_state):
        """
        Function to find a path from a state to other state using only lambda
        
        Input:
        state: initial state
        
        reach_state: state where we want to reach
        
        Output: One possible path from the state to the reach_state using empty_symbols
        
        """
        self.all_paths_empty_transitions = []

        self.__find_path(state, reach_state, memory = [], anti_loops = [])
        
        self.all_paths_empty_transitions = self.all_paths_empty_transitions[0]

    def __find_path(self, state, reach_state, memory = [], anti_loops = []):
        """
        Private function for find_path
        Recursively gets all the paths from state to reach_state
        
        """
        anti_loops.append(state)
        #case to finish recursivity
        if state == reach_state:
            memory.append([state])
            self.all_paths_empty_transitions.append(memory[:])
            return
        else:
            memory.append((state, self.empty_symbol))
            try:
                #checking if there is transiction with the empty_symbol
                valores = self.delta[state,self.empty_symbol]
                for estado in valores:
                    if estado not in anti_loops:
                        self.__find_path(estado, reach_state, memory, anti_loops)
                    memory.pop()                    
            except:
                pass

# Implementation Example

Disclaimer: I know backtracking cost a lot computational speaking, but I really wanted to do it in that way to know what my code did

## Explaining the parsing

Parsing the line is the same as the code for an automaton, however, what it changes is parsing delta

```
all_possible_transictions = {(state,symbol): ["z0_dead"] for state in variables["q"] for symbol in variables["sigma"] }
            
for symbol in variables["sigma"]:
     all_possible_transictions[("z0_dead",symbol)] = ["z0_dead"]


for values in variables["delta"]:
    key = (values[0],values[1])
    
    if key in all_possible_transictions.keys():
        if all_possible_transictions[key][0] == "z0_dead":
            all_possible_transictions[key].pop()
            all_possible_transictions[key].append(values[2])

        else:
            all_possible_transictions[key].append(values[2])

    else:
        all_possible_transictions[key] = [values[2]]
```

What it does, first create all the posible combinations for each state with each symbol and assign them the value "z0_dead" which is the empty state, we did it that way to prevent future errors with that state.

The in the first loop, we create all the posible transiction from the empty set, which only can go to itself

The last loop is to add, the real values for each state with its respective symbol for each state, we did in this way, so each transiction access to a list of posible recheable states.
if the key is already, then check if the "z0_dead" is the first value, that mean that actually z0_dead shouldn't be there so we drop it and add the new values.
the else, is to add values which actually where not in all_possible transitions, if this happen is likely to be an unvalid ndfa

## Explaining the method empty transitions

```
word = str(word)
         
self.trampa = False

self.valid_paths = []

self.not_valid_paths = []

paths = []

self.process_word_recursive(word, self.q0 ,paths, showing_backtrack)

return self.trampa

```

This method allow us to access to the recursive and also without this the backtacking would be harder to apply

Mostly, is for defining new variables and reseting attributes, to then access to the recursive function

## Explaining the private method empty transitions recursive

```
memory_inside = memory[:]

memory_inside.append(state)

if showing_backtracking:
    print(paths)
    
try:
    for new_state in self.delta[(state, self.empty_symbol)]:
        
        if new_state not in memory:
            #backtracking
            paths.append(new_state)

            self.set_empty_transictions.add(new_state)
           
            self.empty_transitions_recursive(new_state, memory_inside, paths, showing_backtracking)

            paths.pop()

        else:
           
            paths.append(new_state)

            if showing_backtracking:
                print(paths)

            paths.pop()

except:
    pass
```

This method is to make the backtraking, first try if there exist transictions with the current state and the empty_symbol, if not the function finish.
If there states, first check if it is not in memory, because if is already in memory it will cause and infinte loop, , then it goes again in recursion, to find all the reacheable states using the empty symbol

## Explaining process symbol function
For this we are gonna divide this function in four different cases, we will avoid talking about what we did with the path variable because is for backtracking, it has no more explanation

### Case 1
#### All the reacheable states only consuming the symbol given

```
self.all_paths_for_a_symbol = []

self.all_reacheble_states  = set(self.delta[(state, symbol)])

paths = [(state, symbol)]

paths.append(self.delta[(state, symbol)])

if self.delta[(state, symbol)] != []:
    self.all_paths_for_a_symbol.append(paths[:])
    
```

As said latter, using self.delta([state,symbol]) we get the states that are reachable using only the symbol given and if is not and empty list we add it to the backtracking

### Case 2
#### Case where reach states consuming the symbol and then lambda if possible

```
for new_state in self.delta[(state, symbol)]:

    if new_state != "z0_dead":
        #adding the state and the symbol for transiction
        paths.append((new_state, self.empty_symbol))


        #adding where it can reach after making that transiction
        empty_transitions_list = self.empty_transitions(new_state, False)

        paths.append(empty_transitions_list)

        if empty_transitions_list != []:
            self.all_paths_for_a_symbol.append(paths[:])
        if showing_backtracking:
            print(paths)

        #uptading the set
        self.all_reacheble_states.update(empty_transitions_list)
        paths.pop()
        paths.pop()        
```

In this case, first we obtain the reacheable states after consuming the symbol, and then in each state we get their reacheable states using the empty symbol.
In the for we get all the reacheable states consuming the symbol, and then applying the funcion empty_transitions in each state we get all the posibles states for this case.

### Case 3 and 4
#### 3. Case where first go to each posible states using the empty symbol and then consumes the given symbol
#### 4. Case where first go to each posible states using the empty symbol and then consumes the given symbol in each state, and finally consumes another empty_symbol in each state.

Basically the case 4, is obtaining all the reacheable states using the empty symbol in each state obtained in case 3

```
new_reacheable_states = self.empty_transitions(state, False)
for new_state in new_reacheable_states:

    paths.append((new_state, symbol))
    paths.append(self.delta[(new_state, symbol)])
    
    self.all_reacheble_states.update(self.delta[(new_state, symbol)])

    self.all_paths_for_a_symbol.append(paths[:])
    paths.pop()

    for second_next_state in self.delta[(new_state, symbol)]:
        if second_next_state != "z0_dead":
            paths.append((second_next_state, self.empty_symbol))

            empty_transitions_list = self.empty_transitions(second_next_state, False)


            paths.append(empty_transitions_list)
            
            if empty_transitions_list != []:
                self.all_paths_for_a_symbol.append(paths[:])

            self.all_reacheble_states.update(empty_transitions_list)

            paths.pop()
            paths.pop()

    paths.pop()

lista_reacheable_state = list(self.all_reacheble_states)    

if "z0_dead" in lista_reacheable_state:
    lista_reacheable_state.append(lista_reacheable_state.pop(lista_reacheable_state.index("z0_dead")))

return lista_reacheable_state
```

What first does, gets all the reacheable states, using the empty_symbol, and then go through another loop. Where in the definition of the loop gets all the reacheable states for each state consuming the symbol, and within the loop gets all the reacheable states of each state consuming an empty symbol

## Function process word

```
word = str(word)
         
self.trampa = False

self.valid_paths = []

self.not_valid_paths = []

paths = []

self.process_word_recursive(word, self.q0 ,paths, showing_backtrack)

return self.trampa
```   

This function is used to use the process_word_recursive which allow us to do the backtracking, so that this function is most to reset atributtes and return the atributte self.trampa which tell us if the word is valid or not.

## Function process_word_recursive

```   
try:
    letter = word[0]

except:
    if state in list(self.f):
        self.trampa = True
        
        self.actual_state = state
        
        self.valid_paths.append(paths[:])
        return 

    #unvalid word
    else:
        
        self.not_valid_actual_state = state
        self.not_valid_paths.append(paths[:])
        return


paths.append((state,letter))


for possible_path in self.process_symbol(state, letter, False):
    paths.append(possible_path)

    if possible_path == "z0_dead":
        paths.pop()
        paths.pop()
        return

    self.process_word_recursive(word[1:], possible_path, paths, showing_backtrack)
    paths.pop()
paths.pop()
```       

This function works to reach all the states while processing a word, first check is still are letter to process in the recursion, if there is no more words, then check is the current state belong to the set of final states, if does it then is a valid path , in the other way is unvalid.


However, if it still more symbols to be process, it goes in the loop, where consuming that symbol it gets all the reacheable states,
if the state is z0_dead, is a dead end, if not, it goes recursively but the word to be process is the rest of it

## Implementation Example

#### Creating an object of the class Automaton

In [2]:
ndfa = NonDeterminsiticAutomaton(empty_symbol = "lambda")

#### Using one of its public method to read an automata from a file

In [3]:
ndfa.load_from_file("automaton_examples/automata.txt")

#### Accesing to its attributes

$q$

In [4]:
ndfa.q

['q0', 'q1', 'q2', 'q3']

$\delta$

In [5]:
ndfa.delta

{('q0', '0'): ['q3', 'q1'],
 ('q0', '1'): ['z0_dead'],
 ('q1', '0'): ['q2'],
 ('q1', '1'): ['q1'],
 ('q2', '0'): ['q2'],
 ('q2', '1'): ['q1'],
 ('q3', '0'): ['q3'],
 ('q3', '1'): ['z0_dead'],
 ('z0_dead', '0'): ['z0_dead'],
 ('z0_dead', '1'): ['z0_dead'],
 ('q0', 'lambda'): ['q2'],
 ('q2', 'lambda'): ['q1']}

$\Sigma$

In [6]:
ndfa.sigma

['0', '1']

$q_{0}$

In [7]:
ndfa.q0

'q0'

$f$

In [8]:
ndfa.f

['q2']

#### Example of getting all the empty transition from a state

As you can see the backtracking prints all the reacheable states and the way that it travels

In [9]:
ndfa.empty_transitions("q0")

['q0']
['q0', 'q2']
['q0', 'q2', 'q1']


['q2', 'q1']

In [10]:
ndfa.empty_transitions("q0", False)

['q2', 'q1']

#### Example of getting all the possible reacheable states consuming one symbol in a state

The backtracking works, in this way the final state reach by that path is the last element of the list which is a list of those final recheable states, if it is empty means that using that path you cannot reach any state

In [11]:
ndfa.process_symbol("q0","0")

-------------------------Case where reach a state only consuming the symbol-------------------------
[('q0', '0'), ['q3', 'q1']]
-------------------------Case where reach a state consuming the symbol and then lambda if possible-------------------------
[('q0', '0'), ('q3', 'lambda'), []]
[('q0', '0'), ('q1', 'lambda'), []]
-------------------------Third and Fourth Case-------------------------
[('q0', 'lambda'), ('q2', '0'), ['q2']]
[('q0', 'lambda'), ('q2', '0'), ('q2', 'lambda'), ['q1']]
[('q0', 'lambda'), ('q1', '0'), ['q2']]
[('q0', 'lambda'), ('q1', '0'), ('q2', 'lambda'), ['q1']]


['q1', 'q3', 'q2']

In [12]:
ndfa.process_symbol("q0","0", False)

['q1', 'q3', 'q2']

#### Example of processing a word

In [13]:
ndfa.process_word("1010000", False)

True

##### Activating backtracking

The backtraking works in the following manner, tuples are the transinction made and the value next to the tuple, is the reach state

In [14]:
ndfa.process_word("101", True)

[('q0', '1')]
[('q0', '1'), 'q1']
[('q0', '1'), 'q1', ('q1', '0')]
[('q0', '1'), 'q1', ('q1', '0'), 'q2']
[('q0', '1'), 'q1', ('q1', '0'), 'q2', ('q2', '1')]
[('q0', '1'), 'q1', ('q1', '0'), 'q2', ('q2', '1'), 'q1']
---------------------------Unvalid---------------------------


[('q0', '1'), 'q1', ('q1', '0'), 'q1']
[('q0', '1'), 'q1', ('q1', '0'), 'q1', ('q1', '1')]
[('q0', '1'), 'q1', ('q1', '0'), 'q1', ('q1', '1'), 'q1']
---------------------------Unvalid---------------------------


[('q0', '1'), 'z0_dead']
---------------------------Unvalid---------------------------




False

In [15]:
ndfa.process_word("100", True)

[('q0', '1')]
[('q0', '1'), 'q1']
[('q0', '1'), 'q1', ('q1', '0')]
[('q0', '1'), 'q1', ('q1', '0'), 'q2']
[('q0', '1'), 'q1', ('q1', '0'), 'q2', ('q2', '0')]
[('q0', '1'), 'q1', ('q1', '0'), 'q2', ('q2', '0'), 'q2']
---------------------------valid-----------------------


[('q0', '1'), 'q1', ('q1', '0'), 'q2', ('q2', '0'), 'q1']
---------------------------Unvalid---------------------------


[('q0', '1'), 'q1', ('q1', '0'), 'q1']
[('q0', '1'), 'q1', ('q1', '0'), 'q1', ('q1', '0')]
[('q0', '1'), 'q1', ('q1', '0'), 'q1', ('q1', '0'), 'q2']
---------------------------valid-----------------------


[('q0', '1'), 'q1', ('q1', '0'), 'q1', ('q1', '0'), 'q1']
---------------------------Unvalid---------------------------


[('q0', '1'), 'z0_dead']
---------------------------Unvalid---------------------------




True

##### Accesing all posible paths

In [16]:
ndfa.valid_paths

[[('q0', '1'), 'q1', ('q1', '0'), 'q2', ('q2', '0'), 'q2'],
 [('q0', '1'), 'q1', ('q1', '0'), 'q1', ('q1', '0'), 'q2']]

#### Printing Path

What printing does, is each enumare line is what the path it takes consuming the symbol in the position enumarete of the word processed, so that each line represents the path of each symbol of the processed word

The only thing that the print_valid_path does not do is all the lambda transiction to get a place, for example, in the chunk belowe ('q0','lambda') it may arrive first to 'q1' and then apply lambda to get 'q2', however, the transictions is summarise

In [17]:
ndfa.print_valid_path()

1.- ('q0', 'lambda')  ======> q2---- ('q2', '1')  ======> q1---- 
2.- ('q1', '0')  ======> q2---- 
3.- ('q2', '0')  ======> q2---- 


#### Function str

In [18]:
str(ndfa)

"Grammar: \n q: ['q0', 'q1', 'q2', 'q3'] \n sigma: ['0', '1'], delta: {('q0', '0'): ['q3', 'q1'], ('q0', '1'): ['z0_dead'], ('q1', '0'): ['q2'], ('q1', '1'): ['q1'], ('q2', '0'): ['q2'], ('q2', '1'): ['q1'], ('q3', '0'): ['q3'], ('q3', '1'): ['z0_dead'], ('z0_dead', '0'): ['z0_dead'], ('z0_dead', '1'): ['z0_dead'], ('q0', 'lambda'): ['q2'], ('q2', 'lambda'): ['q1']}, q0: q0, f: ['q2'], current_sate: q2, empty_symbol: lambda"

## Implementation 2

### Note: this example, it is the most useful to break the code because almost all people do not think in the case of the reacheable states, consuming first the empty symbol, following by the symbol and then again an empty symbol, if it does not process the word "01" they did not think about this case

In [19]:
ndfa2 = NonDeterminsiticAutomaton(empty_symbol = "lambda")
ndfa2.load_from_file("automaton_examples/automata4.txt")

In [20]:
ndfa2.process_symbol("q0","1")

-------------------------Case where reach a state only consuming the symbol-------------------------
[('q0', '1'), ['z0_dead']]
-------------------------Case where reach a state consuming the symbol and then lambda if possible-------------------------
-------------------------Third and Fourth Case-------------------------
[('q0', 'lambda'), ('q1', '1'), ['q2']]
[('q0', 'lambda'), ('q1', '1'), ('q2', 'lambda'), ['q3']]


['q2', 'q3', 'z0_dead']

In [21]:
ndfa2.process_word("01")

[('q0', '0')]
[('q0', '0'), 'q1']
[('q0', '0'), 'q1', ('q1', '1')]
[('q0', '0'), 'q1', ('q1', '1'), 'q2']
---------------------------Unvalid---------------------------


[('q0', '0'), 'q1', ('q1', '1'), 'q3']
---------------------------valid-----------------------


[('q0', '0'), 'q0']
[('q0', '0'), 'q0', ('q0', '1')]
[('q0', '0'), 'q0', ('q0', '1'), 'q2']
---------------------------Unvalid---------------------------


[('q0', '0'), 'q0', ('q0', '1'), 'q3']
---------------------------valid-----------------------


[('q0', '0'), 'q0', ('q0', '1'), 'z0_dead']
---------------------------Unvalid---------------------------


[('q0', '0'), 'z0_dead']
---------------------------Unvalid---------------------------




True

In [22]:
ndfa2.print_valid_path()

1.- ('q0', '0')  ======> q0---- ('q0', 'lambda')  ======> q1---- 
2.- ('q1', '1')  ======> q2---- ('q2', 'lambda')  ======> q3---- 


## Implementation 3

In [29]:
ndfa3 = NonDeterminsiticAutomaton(empty_symbol = "lambda")
ndfa3.load_from_file("automaton_examples/automata3.txt")

In [30]:
ndfa3.process_symbol("q0","1")

-------------------------Case where reach a state only consuming the symbol-------------------------
[('q0', '1'), ['z0_dead']]
-------------------------Case where reach a state consuming the symbol and then lambda if possible-------------------------
-------------------------Third and Fourth Case-------------------------
[('q0', 'lambda'), ('q3', '1'), ['z0_dead']]
[('q0', 'lambda'), ('q2', '1'), ['q1']]
[('q0', 'lambda'), ('q2', '1'), ('q1', 'lambda'), ['q4', 'q2', 'q3', 'q0']]
[('q0', 'lambda'), ('q1', '1'), ['q1']]
[('q0', 'lambda'), ('q1', '1'), ('q1', 'lambda'), ['q4', 'q2', 'q3', 'q0']]
[('q0', 'lambda'), ('q4', '1'), ['z0_dead']]


['q2', 'q3', 'q4', 'q1', 'q0', 'z0_dead']

In [31]:
ndfa3.process_word("11")

[('q0', '1')]
[('q0', '1'), 'q2']
[('q0', '1'), 'q2', ('q2', '1')]
[('q0', '1'), 'q2', ('q2', '1'), 'q2']
---------------------------valid-----------------------


[('q0', '1'), 'q2', ('q2', '1'), 'q3']
---------------------------Unvalid---------------------------


[('q0', '1'), 'q2', ('q2', '1'), 'q4']
---------------------------Unvalid---------------------------


[('q0', '1'), 'q2', ('q2', '1'), 'q1']
---------------------------Unvalid---------------------------


[('q0', '1'), 'q2', ('q2', '1'), 'q0']
---------------------------Unvalid---------------------------


[('q0', '1'), 'q2', ('q2', '1'), 'z0_dead']
---------------------------Unvalid---------------------------


[('q0', '1'), 'q3']
[('q0', '1'), 'q3', ('q3', '1')]
[('q0', '1'), 'q3', ('q3', '1'), 'q2']
---------------------------valid-----------------------


[('q0', '1'), 'q3', ('q3', '1'), 'q3']
---------------------------Unvalid---------------------------


[('q0', '1'), 'q3', ('q3', '1'), 'q4']
------------------------

True

In [32]:
ndfa3.delta

{('q0', '0'): ['q3', 'q1'],
 ('q0', '1'): ['z0_dead'],
 ('q1', '0'): ['q2'],
 ('q1', '1'): ['q1'],
 ('q2', '0'): ['q2'],
 ('q2', '1'): ['q1'],
 ('q3', '0'): ['q3'],
 ('q3', '1'): ['z0_dead'],
 ('q4', '0'): ['z0_dead'],
 ('q4', '1'): ['z0_dead'],
 ('z0_dead', '0'): ['z0_dead'],
 ('z0_dead', '1'): ['z0_dead'],
 ('q0', 'lambda'): ['q2', 'q1'],
 ('q2', 'lambda'): ['q1'],
 ('q1', 'lambda'): ['q3'],
 ('q3', 'lambda'): ['q0', 'q4']}

In [33]:
ndfa3.valid_paths

[[('q0', '1'), 'q2', ('q2', '1'), 'q2'],
 [('q0', '1'), 'q3', ('q3', '1'), 'q2'],
 [('q0', '1'), 'q1', ('q1', '1'), 'q2'],
 [('q0', '1'), 'q0', ('q0', '1'), 'q2']]

In [38]:
ndfa3.print_valid_path(ndfa3.valid_paths[2])

1.- ('q0', 'lambda')  ======> q2---- ('q2', '1')  ======> q1---- 
2.- ('q1', '1')  ======> q1---- ('q1', 'lambda')  ======> q3---- ('q3', 'lambda')  ======> q0---- ('q0', 'lambda')  ======> q2---- 


# Implementation 4

In [ ]:
ndfa4 = NonDeterminsiticAutomaton(empty_symbol = "lambda")
ndfa4.load_from_file("automaton_examples/automata2.txt")

The reason why it does not work is because the txt it has a transiction to final but the state is called Final, due to that capital letter gives an error

# Implementation 5

In [ ]:
ndfa5 = NonDeterminsiticAutomaton(empty_symbol = "epsilon")
ndfa5.load_from_file("automaton_examples/automata5.txt")

In [ ]:
ndfa5.delta

In [ ]:
ndfa5.process_symbol("A","a")

In [ ]:
ndfa5.process_word("abababa")

This does not a valid path because the final states are unreacheable

In [ ]:
ndfa5.print_valid_path()

# Implementation 6

In [ ]:
ndfa6 = NonDeterminsiticAutomaton(empty_symbol = "lambda")
ndfa6.load_from_file("automaton_examples/automata6.txt")

In [ ]:
ndfa6.process_symbol("q0","1")

In [ ]:
ndfa6.process_word("00")

In [ ]:
ndfa6.print_valid_path()

# Implementation 7

In [ ]:
ndfa7 = NonDeterminsiticAutomaton(empty_symbol = "epsilon")
ndfa7.load_from_file("automaton_examples/automata7.txt")

In [ ]:
ndfa7.process_symbol("q0","a")

In [ ]:
ndfa7.process_word("ababaa")

In [ ]:
ndfa7.process_symbol("q1",'a')

In [ ]:
ndfa7.print_valid_path()

# Implementation 8

In [ ]:
ndfa8 = NonDeterminsiticAutomaton(empty_symbol = "epsilon")
ndfa8.load_from_file("automaton_examples/automata8.txt")

In [ ]:
ndfa8.process_word("01")

In [ ]:
ndfa8.process_symbol("q1","1")

In [ ]:
ndfa8.process_word("101")

In [ ]:
ndfa8.print_valid_path()

## Implementation 9

In [ ]:
ndfa9 = NonDeterminsiticAutomaton(empty_symbol = "lambda")
ndfa9.load_from_file("automaton_examples/automata9.txt")
ndfa9.process_word("01", False)

In [ ]:
ndfa9.print_valid_path()

### Preventing Errors

In [ ]:
ndfa.empty_transitions("A")

In [ ]:
ndfa.process_symbol("q0","A")

In [ ]:
ndfa.process_symbol("A","1")

In [ ]:
ndfa.process_word("12A")